In [3]:
import pandas as pd
import re

In [4]:
#space after team name

In [48]:
positions = ['1b','2b', '3b', 'ss', 'rf', 'cf', 'lf', 'p', 'c', 'middle', 'right', 'left', 'center', 'shortstop', '1b.', '2b.', 'ss.', '3b.', 'lf.', 'rf.', 'cf.', 'c.', 'p.']

def quality(count):
    s_count = str(count)
    if len(s_count) >= 6:
                return 1

def remove_dash(play):
    for w in play:
        if len(w) >0:
            if w[-1] == '-':
                w = w[:-1]
    return play

def field_position(play):
    for word in play:
        if word in positions:
            return word 
        
def upper_right(position):
    if (position == 'rf') or (position == 'right') or (position == 'rf.'):
        return 1
    else:
        return 0
    
def upper_center(position):
    if (position == 'cf') or (position == 'middle') or (position == 'center') or (position == 'cf.'):
        return 1
    else:
        return 0
    
def upper_left(position):
    if (position == 'lf') or (position == 'left') or (position == 'lf.'):
        return 1
    else:
        return 0
    
def lower_right(position):
    if (position == '1b') or (position == '2b') or (position == '1b.') or (position == '2b.'):
        return 1
    else:
        return 0
    
def lower_center(position):
    if (position == 'p') or (position == 'c') or (position == 'p.') or (position == 'c.'):
        return 1
    else:
        return 0
    
def lower_left(position):
    if (position == '3b') or (position == 'ss') or (position == 'shortstop') or (position == '3b.') or (position == 'ss.'):
        return 1
    else:
        return 0
    
def runs(end_of_inning):
    if len(end_of_inning) > 0:
        end_list = end_of_inning[0].split(" ")
        runs = end_list[3]
        return runs
    else:
        return 0
    
def hits(end_of_inning):
    if len(end_of_inning) > 0:
        end_list = end_of_inning[0].split(" ")
        hits = end_list[4][2:]
        return hits
    else:
        return 0

def errors(end_of_inning):
    if len(end_of_inning) > 0:
        end_list = end_of_inning[0].split(" ")
        errors = end_list[5][2:]
        return errors
    else:
        return 0
    
def count(count_full):
    if len(count_full) > 0:
        c0 = count_full[0]
        c_list = c0.split(' ')
        c = c_list[0][1:]
        return c
    else: 
        return 0

def pitches(count_full):
    if len(count_full) >= 1:
        c0 = count_full[0]
        c_list = c0.split(' ')
        if len(c_list) > 1:
            c = c_list[1][:-1]
            return c
        else:
            return 0
    else: 
        return 0
    
def make_zero(x):
    if x == []:
        return 0
    else:
        return 1
    
        

In [10]:
import glob

games = glob.glob("*.csv")
len(games)
# game330 = pd.read_csv(games[330])
# game330

500

Read in the csv, clean, and make first columns for play and team. Add a column for the play in a list and find player name by finding first value in the list

In [11]:
test0 = pd.read_csv(games[0])
test0[test0.columns[0]] = test0[test0.columns[0]].apply(lambda x: x.split('\n'))
test0['team_name'] = test0[test0.columns[0]].apply(lambda x: x[0])
test0['play'] = test0[test0.columns[0]].apply(lambda x: x[1])
test0['team_name'] = test0['team_name'].str.strip('title=;\r')
name_play = test0.drop(columns = [test0.columns[0]])

In [12]:
for game in games[1:]:
    test = pd.read_csv(game)
    test[test.columns[0]] = test[test.columns[0]].apply(lambda x: x.split('\n'))
    test['team_name'] = test[test.columns[0]].apply(lambda x: x[0])
    test['play'] = test[test.columns[0]].apply(lambda x: x[1])
    test['team_name'] = test['team_name'].str.strip('title=;\r')
    name_play0 = test.drop(columns = [test.columns[0]])
    name_play = name_play.append(name_play0)


In [13]:
name_play['play'] = name_play['play'].apply(lambda x: x.strip('\r').replace(', ', '-'))
name_play['play_list'] = name_play['play'].apply(lambda x: x.split(' '))
name_play['name'] = name_play['play_list'].apply(lambda x: x[0])

Make new columns for count, the various types of plays, and quality at bats (6 or more pitches)

In [14]:
name_play['count_full'] = name_play['play'].apply(lambda x: re.findall('\(.+\)', x))
name_play['count'] = name_play['count_full'].apply(count)
name_play['pitches'] = name_play['count_full'].apply(pitches)
name_play['pop_up'] = name_play['play'].apply(lambda x: re.findall('popped up to \w+', x)).apply(make_zero)
name_play['K'] = name_play['play'].apply(lambda x: re.findall('struck out swinging', x)).apply(make_zero)
name_play['backwards_K'] = name_play['play'].apply(lambda x: re.findall('struck out looking', x)).apply(make_zero)
name_play['home_run'] = name_play['play'].apply(lambda x: re.findall('homered to \w+ \w+', x)).apply(make_zero)
name_play['fly_ball'] = name_play['play'].apply(lambda x: re.findall('flied out to \w+', x)).apply(make_zero)
name_play['ground_out'] = name_play['play'].apply(lambda x: re.findall('grounded out to \w+', x)).apply(make_zero)
name_play['lined_out'] = name_play['play'].apply(lambda x: re.findall('lined out to \w+', x)).apply(make_zero)
name_play['fouled_out'] = name_play['play'].apply(lambda x: re.findall('fouled out to \w+', x)).apply(make_zero)
name_play['single'] = name_play['play'].apply(lambda x: re.findall('singled', x)).apply(make_zero)
name_play['doubled'] = name_play['play'].apply(lambda x: re.findall('doubled', x)).apply(make_zero)
name_play['tripled'] = name_play['play'].apply(lambda x: re.findall('lined out to \w+', x)).apply(make_zero)
name_play['reached'] = name_play['play'].apply(lambda x: re.findall('reached on', x)).apply(make_zero)
name_play['walked'] = name_play['play'].apply(lambda x: re.findall('walked', x)).apply(make_zero)
name_play['steal'] = name_play['play'].apply(lambda x: re.findall('stole', x)).apply(make_zero)
name_play['scoring'] = name_play['play'].apply(lambda x: x.split(';')).apply(lambda x: x[1:])
name_play['quality_ab'] = name_play['pitches'].apply(quality)
name_play['play_list'] = name_play['play_list'].apply(remove_dash)

In [15]:
name_play['end of inning'] = name_play['play'].apply(lambda x: re.findall('Batting ends R: \d+ H:\d+ E:\d+', x))

Find the position in play_list (the play split on spaces) and assign to right, left, or center

In [53]:
name_play['field_position'] = name_play['play_list'].apply(field_position)
name_play['upper_right'] = name_play['field_position'].apply(upper_right)
name_play['upper_left'] = name_play['field_position'].apply(upper_left)
name_play['upper_center'] = name_play['field_position'].apply(upper_center)
name_play['lower_right'] = name_play['field_position'].apply(lower_right)
name_play['lower_left'] = name_play['field_position'].apply(lower_left)
name_play['lower_center'] = name_play['field_position'].apply(lower_center)
name_play["Runs"] = name_play['end of inning'].apply(runs)
name_play["Hits"] = name_play['end of inning'].apply(hits)
name_play["Errors"] = name_play['end of inning'].apply(errors)
name_play[name_play['team_name'] == 'Rutgers '].T

,0,1,2,3,11,12,13,14,15,16,...,94,95,97,98,99,100,101,102,103,104
team_name,Rutgers,Rutgers,Rutgers,Rutgers,Rutgers,Rutgers,Rutgers,Rutgers,Rutgers,Rutgers,...,Rutgers,Rutgers,Rutgers,Rutgers,Rutgers,Rutgers,Rutgers,Rutgers,Rutgers,Rutgers
play,Hughes-J flied out to cf (3-2 FBFFFBBF).,Marra-A struck out swinging (1-2 BFSS).,Iliopoulos struck out swinging (2-2 SBSBS).,Batting ends R: 0 H:0 E:0,Batting starts,Wingert-K singled up the middle (0-2 SF).,Bowman-N struck out swinging (0-2 FFS).,Cole-M popped up to c-bunt (0-0).,Anico-N grounded out to c unassisted (0-0).,Batting ends R: 0 H:1 E:0,...,Iliopoulos pinch hit for Burcar-A.,Iliopoulos singled through the left side; Anic...,Iliopoulos advanced to second on a wild pitch;...,Wingert-K walked.,Marra-A reached on a fielder's choice-RBI; Win...,Moy-M reached on a fielder's choice; Marra-A a...,Manno-T pinch ran for Moy-M.,Bowman-N to dh for Malito-C.,Bowman-N singled to left field-RBI; Manno-T ad...,Batting ends R: 5 H:3 E:4
play_list,"[Hughes-J, flied, out, to, cf, (3-2, FBFFFBBF).]","[Marra-A, struck, out, swinging, (1-2, BFSS).]","[Iliopoulos, struck, out, swinging, (2-2, SBSB...","[Batting, ends, R:, 0, H:0, E:0]","[Batting, starts]","[Wingert-K, singled, up, the, middle, (0-2, SF).]","[Bowman-N, struck, out, swinging, (0-2, FFS).]","[Cole-M, popped, up, to, c-bunt, (0-0).]","[Anico-N, grounded, out, to, c, unassisted, (0...","[Batting, ends, R:, 0, H:1, E:0]",...,"[Iliopoulos, pinch, hit, for, Burcar-A.]","[Iliopoulos, singled, through, the, left, side...","[Iliopoulos, advanced, to, second, on, a, wild...","[Wingert-K, walked.]","[Marra-A, reached, on, a, fielder's, choice-RB...","[Moy-M, reached, on, a, fielder's, choice;, Ma...","[Manno-T, pinch, ran, for, Moy-M.]","[Bowman-N, to, dh, for, Malito-C.]","[Bowman-N, singled, to, left, field-RBI;, Mann...","[Batting, ends, R:, 5, H:3, E:4]"
name,Hughes-J,Marra-A,Iliopoulos,Batting,Batting,Wingert-K,Bowman-N,Cole-M,Anico-N,Batting,...,Iliopoulos,Iliopoulos,Iliopoulos,Wingert-K,Marra-A,Moy-M,Manno-T,Bowman-N,Bowman-N,Batting
count_full,[(3-2 FBFFFBBF)],[(1-2 BFSS)],[(2-2 SBSBS)],[],[],[(0-2 SF)],[(0-2 FFS)],[(0-0)],[(0-0)],[],...,[],[],[],[],[],[],[],[],[],[]
count,3-2,1-2,2-2,0,0,0-2,0-2,0-0),0-0),0,...,0,0,0,0,0,0,0,0,0,0
pitches,FBFFFBBF,BFSS,SBSBS,0,0,SF,FFS,0,0,0,...,0,0,0,0,0,0,0,0,0,0
pop_up,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
K,0,1,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
backwards_K,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [51]:
name_play.to_csv('data/b10_dataframe.csv')

In [50]:
name_play['field_position'].value_counts()

ss           2285
left         2140
3b           2125
2b           2058
p            2039
lf           1496
1b           1456
right        1354
cf           1196
rf           1190
c            1152
center        721
ss.           557
rf.           540
3b.           480
2b.           476
middle        468
lf.           415
shortstop     349
p.            339
cf.           335
1b.           237
c.            117
Name: field_position, dtype: int64

In [52]:
name_play['play_list'][1]

1                                    [Dunlap, to, 3b.]
1       [Blair, singled, to, left, field, (2-1, BKB).]
1                            [Lambert, stole, second.]
1    [Jefferson, reached, on, a, fielding, error, b...
1        [Holscher, singled, to, right, field, (0-2).]
                           ...                        
1                            [Lambert, stole, second.]
1    [H., MONTEER, grounded, out, to, 3b, (0-2, SSF).]
1    [T., Okada, advanced, to, second, on, a, wild,...
1              [Becker, grounded, out, to, 2b, (0-0).]
1    [Partain, grounded, out, to, 3b-SAC-bunt, (1-1...
Name: play_list, Length: 500, dtype: object